In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import timeit
import datetime
import pprint
import itertools
import pickle
import sklearn
import dask
import dask.dataframe as dd
import dask.array as da
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
import fastparquet
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

In [2]:
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq")\
.to_pandas()
dup_pats = pd.read_pickle("RawData/Cleaned/duplicate_pattext_0712.pkl").tolist()
# Get relevant US Patents
pdf = pdf.loc[~pdf["patent"].isin(dup_pats)]
# Add MSA subfield
pdf["msa_pc"] = list(zip(pdf["inv_msa"], pdf["primclass"]))

In [14]:
# Get number of primary class patents by MSA, and year of their first appearance
p2 = pdf.groupby(["inv_msa", "primclass"]).size()
# Only get primary classes with more than 50 patents
p2 = p2.loc[p2 >= 50].reset_index().rename(columns={0:"size"})
# Date of first appearance
p3 = pdf[["inv_msa", "primclass", "gyear"]].sort_values(["inv_msa", "primclass", "gyear"]).drop_duplicates(["inv_msa", "primclass"])
# Merge
p2 = p2.merge(p3, how="left", on=["inv_msa", "primclass"]).rename(columns={"gyear": "first_gyear"})

# New technology fields after 1990
p3 = p2.loc[p2["first_gyear"] >= 1990]

# Subfield
p3["msa_pc"] = list(zip(p3["inv_msa"], p3["primclass"]))

# Add 5 years to first grant year
p3["first_gyear_5"] = p3["first_gyear"]+5

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
p3

,inv_msa,primclass,size,first_gyear,msa_pc,first_gyear_5,year_group
110,"Albany-Schenectady-Troy, NY",705.0,61,1997,"(Albany-Schenectady-Troy, NY, 705.0)",2002,2000-05
132,"Algona, IA",800.0,57,1992,"(Algona, IA, 800.0)",1997,1995-00
180,"Ames, IA",800.0,221,1991,"(Ames, IA, 800.0)",1996,1995-00
211,"Ann Arbor, MI",385.0,69,1992,"(Ann Arbor, MI, 385.0)",1997,1995-00
240,"Arkadelphia, AR",418.0,57,1998,"(Arkadelphia, AR, 418.0)",2003,2000-05
247,"Atlanta-Sandy Springs-Marietta, GA",1.0,102,1993,"(Atlanta-Sandy Springs-Marietta, GA, 1.0)",1998,1995-00
264,"Atlanta-Sandy Springs-Marietta, GA",124.0,50,1990,"(Atlanta-Sandy Springs-Marietta, GA, 124.0)",1995,1995-00
317,"Atlanta-Sandy Springs-Marietta, GA",382.0,124,1990,"(Atlanta-Sandy Springs-Marietta, GA, 382.0)",1995,1995-00
359,"Atlanta-Sandy Springs-Marietta, GA",707.0,189,1992,"(Atlanta-Sandy Springs-Marietta, GA, 707.0)",1997,1995-00
360,"Atlanta-Sandy Springs-Marietta, GA",709.0,596,1992,"(Atlanta-Sandy Springs-Marietta, GA, 709.0)",1997,1995-00


In [15]:
# Primary class random sample
ns = fastparquet.ParquetFile("DataStore/2018-07-P2/Reg0726/reg_naics_name_sim_tr_0726.parq").to_pandas()
ns["op_msa_pc"] = list(zip(ns["op_inv_msa"], ns["op_primclass"]))
ns["tp_msa_pc"] = list(zip(ns["tp_inv_msa"], ns["tp_primclass"]))
ns = ns.loc[ns["tp_msa_pc"].isin(p3["msa_pc"])]

In [18]:
# All
ns[["norm_sim_docvecs", "norm_sim_mean_docvecs_pc", "norm_sim_mean_docvecs_pc_msa", 
    "inv_msa_match", "primclass_match"]].groupby(["inv_msa_match", "primclass_match"]).mean()

norm_sim_docvecs  norm_sim_mean_docvecs_pc  \
inv_msa_match primclass_match                                               
False         False                   -0.029764                 -0.040592   
              True                     0.387873                  1.687828   
True          False                    0.017407                  0.068526   
              True                     0.617755                  1.687828   

                               norm_sim_mean_docvecs_pc_msa  
inv_msa_match primclass_match                                
False         False                                0.093861  
              True                                 1.235009  
True          False                                0.318236  
              True                                 2.035944

In [19]:
# Similar subfields
ns.loc[(ns["norm_sim_mean_docvecs_pc_msa"] >= 1.5), ["norm_sim_docvecs", "norm_sim_mean_docvecs_pc", "norm_sim_mean_docvecs_pc_msa", 
    "inv_msa_match", "primclass_match"]].groupby(["inv_msa_match", "primclass_match"]).mean()

norm_sim_docvecs  norm_sim_mean_docvecs_pc  \
inv_msa_match primclass_match                                               
False         False                    0.419753                  1.133701   
              True                     0.406325                  1.687828   
True          False                    0.570939                  1.066852   
              True                     0.620576                  1.687828   

                               norm_sim_mean_docvecs_pc_msa  
inv_msa_match primclass_match                                
False         False                                1.570899  
              True                                 1.723883  
True          False                                1.608456  
              True                                 2.035944